<a href="https://colab.research.google.com/github/navneetkrc/fastai_v2/blob/master/loan_tabular_fastai_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fastai v2 Starter Code

## Grabbing the Library

Download for compatibility with colab

In [0]:
import os
!pip install -q fastai2 fastcore feather-format kornia pyarrow wandb nbdev fastprogress --upgrade 
!pip install torchvision===0.4.2
!pip install Pillow==6.2.1 --upgrade
!pip install torch==1.3.1
os._exit(00)

Requirement already up-to-date: Pillow==6.2.1 in /usr/local/lib/python3.6/dist-packages (6.2.1)


We're going to need a variety of imports, most importantly the `tabular.core` module for building the dataset (the rest deal with training the model)

In [0]:
from fastai2.data.all import *
from fastai2.tabular.core import *
from fastai2.tabular.model import *
from fastai2.tabular.all import *
from fastai2.optimizer import *
from fastai2.learner import *
from fastai2.metrics import *
from fastai2.callback.all import *

## Setting Up Our Data

load raw csv data from my github repo

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/navneetkrc/fastai_v2/master/loan_data.csv',error_bad_lines=False, index_col=0)
train.tail()

,activity,borrower_genders,country,country_code,currency_policy,distribution_model,lender_count,loan_amount,original_language,repayment_interval,sector,status,term_in_months,rMPI
55067,Personal Products Sales,male,Nicaragua,NI,not shared,field_partner,6,950,Spanish,bullet,Retail,not_funded,14,0.161
55068,General Store,male,Kenya,KE,shared,field_partner,7,975,English,monthly,Retail,not_funded,14,0.020
55069,Services,female,Indonesia,ID,shared,field_partner,21,925,English,bullet,Services,not_funded,14,0.057
55070,Home Products Sales,female,Tajikistan,TJ,shared,field_partner,34,2775,English,monthly,Retail,not_funded,26,0.033
55071,Dairy,male,Uganda,UG,shared,field_partner,20,1200,English,irregular,Agriculture,not_funded,20,0.346


In [0]:
def remove_outlier(df, col):
  q1 = df[col].quantile(0.05)
  q3 = df[col].quantile(0.95)

  iqr = q3 - q1
  lower_bound  = q1 - (1.5  * iqr)
  upper_bound = q3 + (1.5 * iqr)

  out_df = df.loc[(df[col] > lower_bound) & (df[col] < upper_bound)]
  return out_df

In [4]:
train = remove_outlier(train,'loan_amount')
train.tail()

,activity,borrower_genders,country,country_code,currency_policy,distribution_model,lender_count,loan_amount,original_language,repayment_interval,sector,status,term_in_months,rMPI
55067,Personal Products Sales,male,Nicaragua,NI,not shared,field_partner,6,950,Spanish,bullet,Retail,not_funded,14,0.161
55068,General Store,male,Kenya,KE,shared,field_partner,7,975,English,monthly,Retail,not_funded,14,0.020
55069,Services,female,Indonesia,ID,shared,field_partner,21,925,English,bullet,Services,not_funded,14,0.057
55070,Home Products Sales,female,Tajikistan,TJ,shared,field_partner,34,2775,English,monthly,Retail,not_funded,26,0.033
55071,Dairy,male,Uganda,UG,shared,field_partner,20,1200,English,irregular,Agriculture,not_funded,20,0.346


In [5]:
train.columns

Index(['activity', 'borrower_genders', 'country', 'country_code',
       'currency_policy', 'distribution_model', 'lender_count', 'loan_amount',
       'original_language', 'repayment_interval', 'sector', 'status',
       'term_in_months', 'rMPI'],
      dtype='object')

In [0]:
train.replace({'term_in_months':{'1 Year': 12}},inplace=True)

In [0]:
train.replace({'term_in_months':{'2 Years': 24}},inplace=True)

In [0]:
train.drop(labels=None,axis=0, index=19357, columns=None, level=None, inplace=True, errors='raise')

In [0]:
train['term_in_months']= pd.to_numeric(train['term_in_months'])

In [0]:
train['lender_count']= pd.to_numeric(train['lender_count'])


In [0]:
train['loan_amount']= pd.to_numeric(train['loan_amount'])


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54753 entries, 1 to 55071
Data columns (total 14 columns):
activity              54753 non-null object
borrower_genders      54744 non-null object
country               54753 non-null object
country_code          54753 non-null object
currency_policy       54753 non-null object
distribution_model    54753 non-null object
lender_count          54753 non-null int64
loan_amount           54753 non-null int64
original_language     54753 non-null object
repayment_interval    54753 non-null object
sector                54753 non-null object
status                54753 non-null object
term_in_months        54753 non-null int64
rMPI                  54753 non-null float64
dtypes: float64(1), int64(3), object(10)
memory usage: 6.3+ MB


In [13]:
import pandas_profiling
pandas_profiling.ProfileReport(train)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,15
Number of observations,54753
Total Missing (%),0.0%
Total size in memory,6.3 MiB
Average record size in memory,120.0 B
Numeric,5
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,1


## Making the DataBunch

Next, just like in fastai v1 we need to declare a few things. Specifically our Categorical and Continuous variables, our preprocessors (Normalization, Categorification, and FillMissing), along with how we want to split our data. `fastai` v2 now includes a `RandomSplitter` which is similar to `.split_by_rand_pct()` but now we can specify a custom range for our data (hence `range_of(train)`)

In [0]:
cat_vars = ['activity', 'borrower_genders', 'country', 'country_code',
       'currency_policy', 'distribution_model','repayment_interval',
       'original_language', 'sector', 'status']
cont_vars = ['lender_count','term_in_months', 'rMPI'] #term in month very important
dep_var = 'loan_amount'

In [0]:
procs = [Normalize, Categorify, FillMissing]
splits = RandomSplitter()(range_of(train))

Now that those are defined, we can create a `TabularPandas` object by passing in our dataframe, the `procs`, our variables, what our `y` is, and how we want to split our data. `fastai` v2 is built on a Pipeline structure where first we dictate what we want to do, then we call the databunch (the high-level API is not done yet so we have nothing similar to directly DataBunching an object)

In [16]:
#to = TabularPandas(train, procs, cat_vars, cont_vars, y_names=dep_var, splits=splits)
%time to = TabularPandas(train, procs, cat_vars, cont_vars, dep_var, block_y=TransformBlock(), splits=splits)

CPU times: user 444 ms, sys: 14.2 ms, total: 458 ms
Wall time: 460 ms


If we look at what `to` actually is, we can see what looks to be a bunch of batches of our data aligned into a dataframe that can easily be read!

In [39]:
to

       activity  borrower_genders  ...  term_in_months_na  rMPI_na
52752       102                 3  ...                  1        1
39249       103                 1  ...                  1        1
27786         9                 1  ...                  1        1
29925        71                 2  ...                  1        1
2794         58                 3  ...                  1        1
...         ...               ...  ...                ...      ...
9284         67                 1  ...                  1        1
31500        67                 1  ...                  1        1
19017        70                 3  ...                  1        1
45989       136                 1  ...                  1        1
22121        52                 2  ...                  1        1

[54753 rows x 17 columns]

We can then also easily look at our training and validation datasets by calling `.train` or `.valid`

In [18]:
to.train

       activity  borrower_genders  ...  term_in_months_na  rMPI_na
52752       102                 3  ...                  1        1
39249       103                 1  ...                  1        1
27786         9                 1  ...                  1        1
29925        71                 2  ...                  1        1
2794         58                 3  ...                  1        1
...         ...               ...  ...                ...      ...
24767       133                 3  ...                  1        1
39614        51                 1  ...                  1        1
44622        31                 1  ...                  1        1
53815        50                 3  ...                  1        1
40899        18                 1  ...                  1        1

[43803 rows x 17 columns]

In [19]:
to.valid

       activity  borrower_genders  ...  term_in_months_na  rMPI_na
51200        65                 1  ...                  1        1
16934        79                 3  ...                  1        1
37889        39                 1  ...                  1        1
53178         2                 3  ...                  1        1
12643        38                 1  ...                  1        1
...         ...               ...  ...                ...      ...
9284         67                 1  ...                  1        1
31500        67                 1  ...                  1        1
19017        70                 3  ...                  1        1
45989       136                 1  ...                  1        1
22121        52                 2  ...                  1        1

[10950 rows x 17 columns]

From here we can create our DataBunch object one of two ways. We can either directly do a `dbch = to.dataloaders()`, *or* we can take it one step further and apply custom works to some dataloaders. First let's look at the basic version

In [0]:
dls = to.dataloaders(bs=64)

In [21]:
dls.show_batch()

,activity,borrower_genders,country,country_code,currency_policy,distribution_model,repayment_interval,original_language,sector,status,lender_count_na,term_in_months_na,rMPI_na,lender_count,term_in_months,rMPI,loan_amount
0,Charcoal Sales,female,Kenya,KE,shared,field_partner,irregular,English,Retail,funded,False,False,False,20.000000,14.0,0.075,500.0
1,Personal Medical Expenses,female,Lebanon,LB,not shared,field_partner,monthly,English,Health,funded,False,False,False,44.000001,14.0,0.003,1500.0
2,Food Production/Sales,female,El Salvador,SV,shared,field_partner,monthly,Spanish,Food,funded,False,False,False,24.000000,20.0,0.201,1000.0
3,Restaurant,male,Kenya,KE,shared,field_partner,monthly,English,Food,not_funded,False,False,False,15.000000,14.0,0.020,800.0
4,Fish Selling,female,Philippines,PH,shared,field_partner,irregular,English,Food,not_funded,False,False,False,11.000000,14.0,0.055,600.0
5,Catering,female,Peru,PE,shared,field_partner,monthly,Spanish,Food,funded,False,False,False,23.000000,14.0,0.038,900.0
6,Personal Housing Expenses,female,Nicaragua,NI,shared,field_partner,monthly,Spanish,Housing,funded,False,False,False,14.000000,15.0,0.112,350.0
7,Retail,male,Uganda,UG,shared,field_partner,monthly,English,Retail,not_funded,False,False,False,10.000000,14.0,0.371,1200.0
8,General Store,female,Philippines,PH,shared,field_partner,irregular,English,Retail,funded,False,False,False,13.000000,11.0,0.054,325.0
9,Livestock,male,El Salvador,SV,shared,field_partner,bullet,Spanish,Agriculture,not_funded,False,False,False,23.000000,14.0,0.077,1100.0


##model

In [0]:
max_log_y = np.log(1.2) + np.max(train['loan_amount'])
y_range = (0, max_log_y)

In [0]:
dls.c = 1


In [0]:
#learn = tabular_learner(dls, layers=[200,100], metrics=mae)
learn = tabular_learner(dls, layers=[1000,500], loss_func=MSELossFlat(),
                        config=tabular_config(ps=[0.001,0.01], embed_p=0.04, y_range=y_range), 
                        metrics=mae)

In [40]:
learn.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(154, 27)
    (1): Embedding(4, 3)
    (2): Embedding(61, 16)
    (3): Embedding(61, 16)
    (4): Embedding(3, 3)
    (5): Embedding(2, 2)
    (6): Embedding(4, 3)
    (7): Embedding(9, 5)
    (8): Embedding(16, 8)
    (9): Embedding(3, 3)
    (10): Embedding(2, 2)
    (11): Embedding(2, 2)
    (12): Embedding(2, 2)
  )
  (emb_drop): Dropout(p=0.04, inplace=False)
  (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(95, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.001, inplace=False)
      (2): Linear(in_features=95, out_features=1000, bias=False)
      (3): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.01, inplace=False)
      (2): Linear(in_features=1000, out_features=50

In [41]:
learn.lr_find()

In [29]:
learn.fit_one_cycle(5, 3e-3, wd=0.2)

epoch,train_loss,valid_loss,mae,time
0,169704.656250,159368.718750,278.929138,00:14
1,155110.265625,147550.468750,276.729523,00:15
2,138167.796875,118395.593750,223.724243,00:15
3,119107.718750,112985.203125,206.771820,00:16
4,116340.179688,110565.312500,209.887207,00:15


In [0]:
learn.recorder.plot_loss(skip_start=1000)

In [30]:
learn.fit(15, 2e-4)


epoch,train_loss,valid_loss,mae,time
0,113628.156250,107384.101562,209.378357,00:13
1,105523.515625,95704.796875,196.175934,00:13
2,86828.570312,80721.039062,174.091354,00:13
3,76428.953125,74081.929688,159.290573,00:14
4,78565.250000,74595.546875,158.973785,00:14
5,77228.546875,78314.960938,159.330673,00:13
6,73033.125000,77033.289062,157.812393,00:13
7,83193.742188,76865.585938,157.399124,00:13
8,65044.621094,76777.460938,155.123215,00:14
9,84306.968750,74954.593750,159.794708,00:13
